# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
https://python.langchain.com/docs/modules/agents/agent_types/

In [3]:
import os

# 更换为自己的 Serp API KEY
os.environ["SERPAPI_API_KEY"] = "404001578e14b5b0228dea0c19f1882390ba8652c88d2342cea9e4f14a45c7f6"
os.environ["OPENAI_API_KEY"] = "your key"

In [4]:
from langchain_openai import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0)

In [5]:
# 实例化查询工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [6]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.invoke(
    "成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output:  No. 
Final answer: 第28届大运会 (The 28th Universiade)

In [11]:
# 实际运行 Agent，查询问题（有歧义）
self_ask_with_search.run(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output:  No.

In [12]:
# Reason-only 答案（错误）
print(llm("成都举办的大运会是第几届大运会？"))

/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




成都举办的大运会是第31届大运会。


In [13]:
# Reason-only 答案（错误）
print(llm("2023年大运会举办地在哪里？"))



2023年大运会的举办地将在中国的重庆市。


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [14]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [15]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [16]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.run(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Yes.
Follow up: What is the host city for the 2023 Universiade (大运会)?
Intermediate answer: ['Host city, Yekaterinburg, Russia ; Motto, For You ; Sport, 18 ; Events, 247 ; Opening, 19 July (cancelled).', "Ahead of the start of this year's Summer Universiade in Napoli, FISU held its biannual two-day Executive Committee meeting in the coastal Italian city, ...", 'From 16 to 27 July 2025, the cities of Bochum, Duisburg, Düsseldorf, Essen and Mülheim an der Ruhr will join together to host a programme set to combine ...', 'The most recent summer event was the 2021 Summer World University Games held in Chengdu, China from 28 July – 8 August 2023, after being postponed three times ...', '每日一词∣成都大运会主媒体中心Main Media Center of Chengdu Universiade ... host city. 全民健身活动. public fitness initiatives. 竞技体育.', 'This marked the first time the FISU Games Winter returned to a former host city as Lake Placid held the event in 1972, making it the only location outside 

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output: The final answer is: Yekaterinburg, Russia